# Using EcoFOCIpy to process raw field data

## CF1601 ( - Cape Flattery - niskin and rosette values are not equivalent... key match so niskin is whats reported)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [27]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [28]:
sample_data_dir = '/Users/bell/ecoraid/2016/CTDcasts/cf1601/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [29]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'CF1601' #no hyphens
cruise_meta_file = sample_data_dir+'logs/CF1601.yaml'
inst_meta_file = sample_data_dir+'logs/EMA_standard_CTDwWetLabs.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD001.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD002.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD003.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD004.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD005.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD006.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD007.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD008.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD009.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD010.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD011.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD012.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD013.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD028.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD029.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD030.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD031.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD032.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD033.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD034.btl
Processing /Users/bell/ecoraid/2016/CTDcasts/cf1601/rawconverted/CTD035.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [30]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['CTD017.btl'].describe()

,sal00,sal11,sbeox0mm/kg,sbeox0ps,sbeox1mm/kg,sigma-t00,density11,prdm,depsm,t090c,t190c,dz/dtm,wetstar,obs,cstarat0,cstartr0,scan,datetime
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4
mean,31.604500,31.603625,288.925500,96.238750,295.964250,24.704325,1024.768425,14.023250,13.897500,7.096200,7.099075,-0.055500,1.776125,50.315250,0.502025,88.233400,7216.00000,2016-09-02 03:48:37.249999872
min,31.461500,31.464400,284.926000,88.519000,292.218000,24.344900,1024.359300,1.082000,1.073000,3.545300,3.557200,-0.137000,0.443100,50.225500,0.392800,85.267000,6052.00000,2016-09-02 03:47:49
25%,31.471025,31.471300,285.501250,95.719750,292.308750,24.352250,1024.383375,7.610750,7.542500,5.979875,5.980750,-0.101000,0.849975,50.248000,0.411175,86.510350,6652.00000,2016-09-02 03:48:13.750000128
50%,31.548350,31.548400,287.098000,98.570000,294.274500,24.570500,1024.636250,15.062500,14.927000,7.907600,7.906300,-0.079000,1.439750,50.275400,0.488900,88.509900,7191.50000,2016-09-02 03:48:36
75%,31.681825,31.680725,290.522250,99.089000,297.930000,24.922575,1025.021300,21.475000,21.282000,9.023925,9.024625,-0.033500,2.365900,50.342650,0.579750,90.232950,7755.50000,2016-09-02 03:48:59.500000
max,31.859800,31.853300,296.580000,99.296000,303.090000,25.331400,1025.441900,24.886000,24.663000,9.024300,9.026500,0.073000,3.781900,50.484700,0.637500,90.646800,8429.00000,2016-09-02 03:49:28
std,0.185229,0.181669,5.329632,5.170937,5.099462,0.465964,0.508526,10.697308,10.601061,2.590689,2.585961,0.090294,1.465024,0.116528,0.116735,2.569177,1009.61808,NaN


## Create BTL report file

In [31]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd017':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [32]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'CF1601',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'Cape Flattery',
 'ShipID': 'CF',
 'StartDate': datetime.date(2016, 8, 28),
 'EndDate': datetime.date(2016, 9, 12),
 'Project': 'EMA',
 'ChiefScientist': '',
 'StartPort': '',
 'EndPort': '',
 'CruiseLocation': 'Bering Sea',
 'Description': '',
 'CruiseYear': 2016,
 'ctdlogs_pdf_name': 'cf1601_CastLogs.pdf',
 'Accession': '',
 'Links': '\r',
 'QC_Status': '',
 'QC_Signature': ''}

In [33]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD017']

{'id': 22133,
 'Vessel': 'Cape Flattery',
 'CruiseID': 'CF1601',
 'Project_Leg': '',
 'UniqueCruiseID': 'CF1601',
 'Project': '',
 'StationNo_altname': '17',
 'ConsecutiveCastNo': 'CTD017',
 'LatitudeDeg': 62,
 'LatitudeMin': 30.1,
 'LongitudeDeg': 170,
 'LongitudeMin': 0.25,
 'GMTDay': 2,
 'GMTMonth': 'sep',
 'GMTYear': 2016,
 'GMTTime': 13620,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': 9.05,
 'BottomDepth': 32,
 'StationNameID': '',
 'MaxDepth': 25,
 'InstrumentSerialNos': 'Press SN = 0775, Pri Temp SN = 4354, Sec Temp SN = 4394, Pri Cond SN = 2879, Sec Cond Sn = 3072, Backscatterance = 2615, Fluor Sn = WS3S-994P, Transmssometer = Wetlabs CST-676PR pri O2 Sn = 0804, sec O2 Sn = 1301',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;3;4;10',
 'NutrientBtlNumbers': '62;63;64;65',
 'OxygenBtlNiskinNo

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [34]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [35]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD017.btl'] = cruise_data['CTD017.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'attenuation',
                            'cstartr0':'transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['CTD017.btl'].sample()

,salinity_ch1,salinity_ch2,sbeox0mm/kg,oxy_percentsat_ch1,sbeox1mm/kg,sigma_t_ch1,density11,pressure,depsm,temperature_ch1,temperature_ch2,dz/dtm,wetstar,obs,attenuation,transmittance,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
4.0,31.6225,31.6232,296.58,98.12,303.09,24.7863,1024.8811,20.338,20.155,6.7914,6.7886,0.073,3.7819,50.2953,0.5605,86.9248,6852.0,2016-09-02 03:48:22,ctd017


In [36]:
cruise_data['CTD017.btl'].columns

Index(['salinity_ch1', 'salinity_ch2', 'sbeox0mm/kg', 'oxy_percentsat_ch1',
       'sbeox1mm/kg', 'sigma_t_ch1', 'density11', 'pressure', 'depsm',
       'temperature_ch1', 'temperature_ch2', 'dz/dtm', 'wetstar', 'obs',
       'attenuation', 'transmittance', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [37]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [38]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD017.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [39]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 640B
Dimensions:             (bottle: 4)
Coordinates:
  * bottle              (bottle) float64 32B 10.0 4.0 3.0 1.0
Data variables: (12/19)
    salinity_ch1        (bottle) float64 32B 31.86 31.62 31.46 31.47
    salinity_ch2        (bottle) float64 32B 31.85 31.62 31.46 31.47
    sbeox0mm/kg         (bottle) float64 32B 288.5 296.6 284.9 285.7
    oxy_percentsat_ch1  (bottle) float64 32B 88.52 98.12 99.02 99.3
    sbeox1mm/kg         (bottle) float64 32B 296.2 303.1 292.2 292.3
    sigma_t_ch1         (bottle) float64 32B 25.33 24.79 24.34 24.35
    ...                  ...
    obs                 (bottle) float64 32B 50.48 50.3 50.26 50.23
    attenuation         (bottle) float64 32B 0.6375 0.5605 0.4173 0.3928
    transmittance       (bottle) float64 32B 85.27 86.92 90.09 90.65
    scan                (bottle) float64 32B 6.052e+03 6.852e+03 ... 8.429e+03
    datetime            (bottle) datetime64[ns] 32B 2016-09-02T03:47:49 ... 2...
    cast                (bottle) object 32B 'ctd017' 'ctd017' 'ctd017' 'ctd017'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [40]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [41]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD017.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [42]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 344B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 4)
Coordinates:
  * bottle              (bottle) float64 32B 10.0 4.0 3.0 1.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    salinity_ch1        (latitude, longitude, time, bottle) float64 32B 31.86...
    salinity_ch2        (latitude, longitude, time, bottle) float64 32B 31.85...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 32B 88.52...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 32B 25.33...
    pressure            (latitude, longitude, time, bottle) float64 32B 24.89...
    temperature_ch1     (latitude, longitude, time, bottle) float64 32B 3.545...
    temperature_ch2     (latitude, longitude, time, bottle) float64 32B 3.557...
    attenuation         (latitude, longitude, time, bottle) float64 32B 0.637...
    transmittance       (latitude, longitude, time, bottle) float64 32B 85.27...

In [43]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD017')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 344B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 4)
Coordinates:
  * bottle              (bottle) float64 32B 10.0 4.0 3.0 1.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    salinity_ch1        (latitude, longitude, time, bottle) float64 32B 31.86...
    salinity_ch2        (latitude, longitude, time, bottle) float64 32B 31.85...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 32B 88.52...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 32B 25.33...
    pressure            (latitude, longitude, time, bottle) float64 32B 24.89...
    temperature_ch1     (latitude, longitude, time, bottle) float64 32B 3.545...
    temperature_ch2     (latitude, longitude, time, bottle) float64 32B 3.557...
    attenuation         (latitude, longitude, time, bottle) float64 32B 0.637...
    transmittance       (latitude, longitude, time, bottle) float64 32B 85.27...
Attributes: (12/48)
    CruiseID:                      CF1601
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        Cape Flattery
    ShipID:                        CF
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-04-08T11:16:22Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

<div class="warning" style='background-color:#ffcccb; color: #FF0000; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>WARNING</b></p>
<p style='margin-left:1em;'>bottle/niskin and rosette position should be the same but can be different (example, bottles are labeled sequentially but a rosette position is skipped due to balancing or other instruments.  On this cruise the following rosette positin was fired, while the bottles where labled differently</p>
cruise_data[cast]
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"><span style="font-weight:bold">Rosette</span></th>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">4</th>
    <th class="tg-0lax">10</th>
    <th class="tg-0lax">11</th>
    <th class="tg-0lax">12</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax"><span style="font-weight:bold">Niskin</span><br></td>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">6</td>
  </tr>
</tbody>
</table>
</div>

In [44]:
import pandas as pd
nisk_btl = pd.DataFrame(data=[[1,1],[2,3],[3,4],[4,10],[5,11],[6,12]],columns=['bottle','rosette']) 
for cast in cruise_data.keys():
    cruise_data[cast] = pd.merge(nisk_btl, 
                                 cruise_data[cast], 
                                 left_on='rosette', 
                                 right_on='bottle').drop('rosette',axis=1).set_index('bottle')      

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [45]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'attenuation',
                            'cstartr0':'transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history